In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from data_wrangling.datamanager import DataLoader as DL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
dl = DL()

In [4]:
sessions = ['s1', 's4', 's5', 's6', 's7', 's10']
data = dl.get_fcx2(sessions)

In [9]:
[d['data'].shape for d in data.values()]

[(1303952, 73),
 (977418, 66),
 (1017999, 172),
 (838999, 91),
 (762999, 110),
 (815892, 110)]

In [4]:
def get_as_array(session):
    arrs = data[session]['data']
    mn = arrs.mean(0).reshape((1, -1))
    st = arrs.std (0).reshape((1, -1))
    arrs -= mn
    arrs /= st
    return arrs

Xs = [
    get_as_array(s)
    for s in sessions
]

data = 0

In [5]:
n_c_max = max(Xs, key = lambda X: X.shape[1]).shape[1]
n_c_max

172

In [6]:
n_channelss = [X.shape[1] for X in Xs]

def pad_with_zeros(X):
    zeros = np.zeros((X.shape[0], n_c_max - X.shape[1]))
    out = np.concatenate([X, zeros], 1)
    return out

Xs = [pad_with_zeros(X) for X in Xs]

In [7]:
[X.shape for X in Xs], n_channelss

([(1303952, 172),
  (977418, 172),
  (1017999, 172),
  (838999, 172),
  (762999, 172),
  (815892, 172)],
 [73, 66, 172, 91, 110, 110])

In [8]:
t_in  = 100

train_frac = 0.5
val_frac   = 0.2

train_ranges = np.array([
    (
        0,
        int(X.shape[0] * train_frac) - t_in
    )
    for X in Xs
])

val_ranges = np.array([
    (
        train_max + t_in, 
        train_max + int(X.shape[0] * val_frac) - t_in
    )
    for (_, train_max), X in zip(train_ranges, Xs)
])

test_ranges = np.array([
    (
        val_max + t_in,
        len(X) - t_in
    )
    for (val_min, val_max), X in zip(val_ranges, Xs)
])

np.concatenate([
    train_ranges,
    val_ranges,
    test_ranges
], 1)

array([[      0,  651876,  651976,  912566,  912666, 1303852],
       [      0,  488609,  488709,  683992,  684092,  977318],
       [      0,  508899,  508999,  712398,  712498, 1017899],
       [      0,  419399,  419499,  587098,  587198,  838899],
       [      0,  381399,  381499,  533898,  533998,  762899],
       [      0,  407846,  407946,  570924,  571024,  815792]])

In [9]:
n_sessions = len(sessions)

def get_random_data_idxs(sess_no, n, mode = 'train'):
    if mode == 'train':
        _idxs = np.arange(*list(train_ranges[sess_no].copy()))
    elif mode == 'val':
        _idxs = np.arange(*list(val_ranges[sess_no].copy()))
    elif mode == 'test':
        _idxs = np.arange(*list(test_ranges[sess_no].copy()))
        
    np.random.shuffle(_idxs)
    return _idxs[:n]

def data_generator(
    batch_size,
    steps_per_epoch,
    epochs,
    mode = 'train'
):
    
    for _ in range(steps_per_epoch * epochs):
        
        # Select session indices
        counts = pd.Series(
            np.random.randint(n_sessions, size = batch_size)
        ).value_counts()

        # Session-wise indices
        idxs = [
            get_random_data_idxs(idx, n, mode = mode)
            for idx, n in enumerate(counts)
        ]

        # Create data
        x = np.concatenate([
            np.stack([
                np.concatenate([
                    X[idx + t], 
                    np.array([n_channels] * len(idx)).reshape((-1, 1))
                ], 1)
                for t in range(t_in) 
            ], 1)
            for n_channels, idx, X in zip(n_channelss, idxs, Xs)
        ])

        y = np.concatenate([
            np.stack([
                X[idx + t]
                for t in range(t_in) 
            ], 1)
            for idx, X in zip(idxs, Xs)
        ])
        
        yield (x, y)

In [10]:
class Splitter(layers.Layer):
    def call(self, inputs):
        signals = inputs[:,:,:-1]
        labels  = tf.cast(inputs[:, :t_in, -1], tf.int32)
        return signals, labels
    
class Combiner(layers.Layer):
    def call(self, inputs):
        outputs, labels = inputs
        
        idxs_for_mask = tf.ones(tf.shape(outputs), dtype = tf.int32) * np.arange(outputs.shape[-1])
        lims_for_mask = tf.repeat(tf.expand_dims(labels, 2), outputs.shape[-1], 2)
        return outputs * tf.cast(idxs_for_mask < lims_for_mask, tf.float32)
    
input_layer = layers.Input((t_in, n_c_max + 1))
signals, labels = Splitter()(input_layer)

encoder = layers.Bidirectional(
    layers.LSTM(64)
)(signals)

repeater = layers.RepeatVector(t_in)(encoder)

decoder1 = layers.LSTM(128, return_sequences = True)(repeater)
decoder2 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True)
)(decoder1)
decoder3 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True)
)(decoder2)

regressor = layers.Dense(n_c_max)(decoder3)
cleaner = Combiner()([regressor, labels])

model = keras.Model(input_layer, cleaner)
model.compile(loss = 'mse', optimizer = 'adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 173)]   0                                            
__________________________________________________________________________________________________
splitter (Splitter)             ((None, 100, 172), ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 128)          121344      splitter[0][0]                   
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 100, 128)     0           bidirectional[0][0]              
______________________________________________________________________________________________

In [11]:
batch_size      = 1024
steps_per_epoch = 80
epochs          = 50

validation_batch_size     = batch_size
vaidation_steps_per_epoch = 20
# Necessary for function
validation_epochs         = 1

hist1 = model.fit(
    # Training data and configuration
    x = data_generator(
        batch_size, 
        steps_per_epoch, 
        epochs
    ),
    batch_size      = batch_size,
    steps_per_epoch = steps_per_epoch,
    epochs          = epochs,
    
    # Validation data
    validation_data = tf.data.Dataset.from_generator(
        lambda: data_generator(
            validation_batch_size, 
            vaidation_steps_per_epoch, 
            validation_epochs,
            mode = 'val'
        ),
        output_signature = (
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max + 1), dtype = tf.float64),
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max),     dtype = tf.float64),
        )
    ),
    validation_batch_size = validation_batch_size,
    validation_steps      = vaidation_steps_per_epoch,
    
    callbacks = [keras.callbacks.EarlyStopping(
        patience = 2,
        min_delta = 1e-3
    )]
)

Epoch 1/50
80/80 [==============================] - 42s 451ms/step - loss: 0.4830 - val_loss: 0.3593
Epoch 2/50
80/80 [==============================] - 32s 399ms/step - loss: 0.3390 - val_loss: 0.3290
Epoch 3/50
80/80 [==============================] - 32s 403ms/step - loss: 0.3079 - val_loss: 0.3013
Epoch 4/50
80/80 [==============================] - 32s 404ms/step - loss: 0.2872 - val_loss: 0.2867
Epoch 5/50
80/80 [==============================] - 32s 401ms/step - loss: 0.2729 - val_loss: 0.2715
Epoch 6/50
80/80 [==============================] - 32s 404ms/step - loss: 0.2591 - val_loss: 0.2618
Epoch 7/50
80/80 [==============================] - 32s 403ms/step - loss: 0.2452 - val_loss: 0.2455
Epoch 8/50
80/80 [==============================] - 32s 405ms/step - loss: 0.2309 - val_loss: 0.2346
Epoch 9/50
80/80 [==============================] - 32s 403ms/step - loss: 0.2162 - val_loss: 0.2210
Epoch 10/50
80/80 [==============================] - 32s 403ms/step - loss: 0.2050 - val_lo

In [12]:
batch_size      = 128
steps_per_epoch = 200
epochs          = 50

validation_batch_size     = batch_size
vaidation_steps_per_epoch = 20
validation_epochs         = 1

hist2 = model.fit(
    # Training data and configuration
    x = data_generator(
        batch_size, 
        steps_per_epoch, 
        epochs
    ),
    batch_size      = batch_size,
    steps_per_epoch = steps_per_epoch,
    epochs          = epochs,
    
    # Validation data
    validation_data = tf.data.Dataset.from_generator(
        lambda: data_generator(
            validation_batch_size, 
            vaidation_steps_per_epoch, 
            validation_epochs,
            mode = 'val'
        ),
        output_signature = (
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max + 1), dtype = tf.float64),
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max),     dtype = tf.float64),
        )
    ),
    validation_batch_size = validation_batch_size,
    validation_steps      = vaidation_steps_per_epoch,
    
    callbacks = [keras.callbacks.EarlyStopping(
        patience = 3,
        min_delta = 5e-4
    )]
)

Epoch 1/50
200/200 [==============================] - 16s 82ms/step - loss: 0.1475 - val_loss: 0.1463
Epoch 2/50
200/200 [==============================] - 15s 76ms/step - loss: 0.1373 - val_loss: 0.1561
Epoch 3/50
200/200 [==============================] - 15s 76ms/step - loss: 0.1329 - val_loss: 0.1425
Epoch 4/50
200/200 [==============================] - 15s 76ms/step - loss: 0.1293 - val_loss: 0.1324
Epoch 5/50
200/200 [==============================] - 15s 76ms/step - loss: 0.1226 - val_loss: 0.1303
Epoch 6/50
200/200 [==============================] - 15s 76ms/step - loss: 0.1208 - val_loss: 0.1243
Epoch 7/50
200/200 [==============================] - 15s 76ms/step - loss: 0.1196 - val_loss: 0.1252
Epoch 8/50
200/200 [==============================] - 15s 77ms/step - loss: 0.1195 - val_loss: 0.1245
Epoch 9/50
200/200 [==============================] - 15s 76ms/step - loss: 0.1166 - val_loss: 0.1217
Epoch 10/50
200/200 [==============================] - 15s 76ms/step - loss: 0.117

In [13]:
mse = model.evaluate(
    x = data_generator(
        1024,
        1000,
        1,
        mode = 'test'
    )
)

1000/1000 [==============================] - 279s 278ms/step - loss: 0.1169


In [14]:
input_layer = layers.Input((t_in, n_c_max + 1))
signals, labels = Splitter()(input_layer)

encoder1 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True),
)(signals)

encoder2 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True),
)(encoder1)

encoder3 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True),
)(encoder2)

regressor = layers.Dense(n_c_max)(encoder3)
cleaner = Combiner()([regressor, labels])

model = keras.Model(input_layer, cleaner)
model.compile(loss = 'mse', optimizer = 'adam')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100, 173)]   0                                            
__________________________________________________________________________________________________
splitter_1 (Splitter)           ((None, 100, 172), ( 0           input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 100, 128)     121344      splitter_1[0][0]                 
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 100, 128)     98816       bidirectional_3[0][0]            
____________________________________________________________________________________________

In [15]:
batch_size      = 1024
steps_per_epoch = 80
epochs          = 50

validation_batch_size     = batch_size
vaidation_steps_per_epoch = 20
# Necessary for function
validation_epochs         = 1

hist1 = model.fit(
    # Training data and configuration
    x = data_generator(
        batch_size, 
        steps_per_epoch, 
        epochs
    ),
    batch_size      = batch_size,
    steps_per_epoch = steps_per_epoch,
    epochs          = epochs,
    
    # Validation data
    validation_data = tf.data.Dataset.from_generator(
        lambda: data_generator(
            validation_batch_size, 
            vaidation_steps_per_epoch, 
            validation_epochs,
            mode = 'val'
        ),
        output_signature = (
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max + 1), dtype = tf.float64),
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max),     dtype = tf.float64),
        )
    ),
    validation_batch_size = validation_batch_size,
    validation_steps      = vaidation_steps_per_epoch,
    
    callbacks = [keras.callbacks.EarlyStopping(
        patience = 2,
        min_delta = 1e-3
    )]
)

Epoch 1/50
80/80 [==============================] - 38s 428ms/step - loss: 0.3890 - val_loss: 0.1870
Epoch 2/50
80/80 [==============================] - 31s 396ms/step - loss: 0.1604 - val_loss: 0.1167
Epoch 3/50
80/80 [==============================] - 31s 394ms/step - loss: 0.1003 - val_loss: 0.0833
Epoch 4/50
80/80 [==============================] - 31s 396ms/step - loss: 0.0711 - val_loss: 0.0626
Epoch 5/50
80/80 [==============================] - 31s 396ms/step - loss: 0.0556 - val_loss: 0.0515
Epoch 6/50
80/80 [==============================] - 31s 394ms/step - loss: 0.0467 - val_loss: 0.0466
Epoch 7/50
80/80 [==============================] - 31s 394ms/step - loss: 0.0397 - val_loss: 0.0405
Epoch 8/50
80/80 [==============================] - 31s 394ms/step - loss: 0.0348 - val_loss: 0.0364
Epoch 9/50
80/80 [==============================] - 31s 390ms/step - loss: 0.0313 - val_loss: 0.0325
Epoch 10/50
80/80 [==============================] - 31s 396ms/step - loss: 0.0288 - val_lo

In [16]:
batch_size      = 128
steps_per_epoch = 200
epochs          = 50

validation_batch_size     = batch_size
vaidation_steps_per_epoch = 20
validation_epochs         = 1

hist2 = model.fit(
    # Training data and configuration
    x = data_generator(
        batch_size, 
        steps_per_epoch, 
        epochs
    ),
    batch_size      = batch_size,
    steps_per_epoch = steps_per_epoch,
    epochs          = epochs,
    
    # Validation data
    validation_data = tf.data.Dataset.from_generator(
        lambda: data_generator(
            validation_batch_size, 
            vaidation_steps_per_epoch, 
            validation_epochs,
            mode = 'val'
        ),
        output_signature = (
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max + 1), dtype = tf.float64),
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max),     dtype = tf.float64),
        )
    ),
    validation_batch_size = validation_batch_size,
    validation_steps      = vaidation_steps_per_epoch,
    
    callbacks = [keras.callbacks.EarlyStopping(
        patience = 3,
        min_delta = 5e-4
    )]
)

Epoch 1/50
200/200 [==============================] - 16s 78ms/step - loss: 0.0154 - val_loss: 0.0254
Epoch 2/50
200/200 [==============================] - 15s 73ms/step - loss: 0.0144 - val_loss: 0.0166
Epoch 3/50
200/200 [==============================] - 15s 73ms/step - loss: 0.0134 - val_loss: 0.0147
Epoch 4/50
200/200 [==============================] - 15s 73ms/step - loss: 0.0127 - val_loss: 0.0132
Epoch 5/50
200/200 [==============================] - 15s 73ms/step - loss: 0.0122 - val_loss: 0.0136
Epoch 6/50
200/200 [==============================] - 15s 73ms/step - loss: 0.0118 - val_loss: 0.0131
Epoch 7/50
200/200 [==============================] - 15s 73ms/step - loss: 0.0114 - val_loss: 0.0119
Epoch 8/50
200/200 [==============================] - 15s 73ms/step - loss: 0.0110 - val_loss: 0.0133
Epoch 9/50
200/200 [==============================] - 15s 73ms/step - loss: 0.0108 - val_loss: 0.0134
Epoch 10/50
200/200 [==============================] - 15s 73ms/step - loss: 0.010

In [17]:
mse = model.evaluate(
    x = data_generator(
        1024,
        1000,
        1,
        mode = 'test'
    )
)

1000/1000 [==============================] - 279s 278ms/step - loss: 0.0114


In [18]:
input_layer = layers.Input((t_in, n_c_max + 1))
signals, labels = Splitter()(input_layer)

encoder1 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True),
)(signals)

encoder2 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True),
)(encoder1)

decoder1 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True)
)(encoder2)
decoder2 = layers.Bidirectional(
    layers.LSTM(64, return_sequences = True)
)(decoder1)

regressor = layers.Dense(n_c_max)(decoder2)
cleaner = Combiner()([regressor, labels])

model = keras.Model(input_layer, cleaner)
model.compile(loss = 'mse', optimizer = 'adam')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100, 173)]   0                                            
__________________________________________________________________________________________________
splitter_2 (Splitter)           ((None, 100, 172), ( 0           input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 100, 128)     121344      splitter_2[0][0]                 
__________________________________________________________________________________________________
bidirectional_7 (Bidirectional) (None, 100, 128)     98816       bidirectional_6[0][0]            
____________________________________________________________________________________________

In [19]:
batch_size      = 1024
steps_per_epoch = 80
epochs          = 50

validation_batch_size     = batch_size
vaidation_steps_per_epoch = 20
# Necessary for function
validation_epochs         = 1

hist1 = model.fit(
    # Training data and configuration
    x = data_generator(
        batch_size, 
        steps_per_epoch, 
        epochs
    ),
    batch_size      = batch_size,
    steps_per_epoch = steps_per_epoch,
    epochs          = epochs,
    
    # Validation data
    validation_data = tf.data.Dataset.from_generator(
        lambda: data_generator(
            validation_batch_size, 
            vaidation_steps_per_epoch, 
            validation_epochs,
            mode = 'val'
        ),
        output_signature = (
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max + 1), dtype = tf.float64),
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max),     dtype = tf.float64),
        )
    ),
    validation_batch_size = validation_batch_size,
    validation_steps      = vaidation_steps_per_epoch,
    
    callbacks = [keras.callbacks.EarlyStopping(
        patience = 2,
        min_delta = 1e-3
    )]
)

Epoch 1/50
80/80 [==============================] - 41s 448ms/step - loss: 0.3890 - val_loss: 0.1951
Epoch 2/50
80/80 [==============================] - 33s 415ms/step - loss: 0.1695 - val_loss: 0.1317
Epoch 3/50
80/80 [==============================] - 32s 401ms/step - loss: 0.1165 - val_loss: 0.0992
Epoch 4/50
80/80 [==============================] - 32s 403ms/step - loss: 0.0877 - val_loss: 0.0789
Epoch 5/50
80/80 [==============================] - 32s 408ms/step - loss: 0.0720 - val_loss: 0.0726
Epoch 6/50
80/80 [==============================] - 32s 405ms/step - loss: 0.0628 - val_loss: 0.0636
Epoch 7/50
80/80 [==============================] - 33s 414ms/step - loss: 0.0548 - val_loss: 0.0547
Epoch 8/50
80/80 [==============================] - 32s 408ms/step - loss: 0.0491 - val_loss: 0.0505
Epoch 9/50
80/80 [==============================] - 32s 404ms/step - loss: 0.0446 - val_loss: 0.0455
Epoch 10/50
80/80 [==============================] - 33s 411ms/step - loss: 0.0417 - val_lo

In [20]:
batch_size      = 128
steps_per_epoch = 200
epochs          = 50

validation_batch_size     = batch_size
vaidation_steps_per_epoch = 20
validation_epochs         = 1

hist2 = model.fit(
    # Training data and configuration
    x = data_generator(
        batch_size, 
        steps_per_epoch, 
        epochs
    ),
    batch_size      = batch_size,
    steps_per_epoch = steps_per_epoch,
    epochs          = epochs,
    
    # Validation data
    validation_data = tf.data.Dataset.from_generator(
        lambda: data_generator(
            validation_batch_size, 
            vaidation_steps_per_epoch, 
            validation_epochs,
            mode = 'val'
        ),
        output_signature = (
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max + 1), dtype = tf.float64),
            tf.TensorSpec(shape = (validation_batch_size, t_in, n_c_max),     dtype = tf.float64),
        )
    ),
    validation_batch_size = validation_batch_size,
    validation_steps      = vaidation_steps_per_epoch,
    
    callbacks = [keras.callbacks.EarlyStopping(
        patience = 3,
        min_delta = 5e-4
    )]
)

Epoch 1/50
200/200 [==============================] - 17s 85ms/step - loss: 0.0253 - val_loss: 0.0241
Epoch 2/50
200/200 [==============================] - 16s 78ms/step - loss: 0.0225 - val_loss: 0.0236
Epoch 3/50
200/200 [==============================] - 16s 78ms/step - loss: 0.0216 - val_loss: 0.0238
Epoch 4/50
200/200 [==============================] - 15s 76ms/step - loss: 0.0207 - val_loss: 0.0222
Epoch 5/50
200/200 [==============================] - 15s 75ms/step - loss: 0.0209 - val_loss: 0.0228
Epoch 6/50
200/200 [==============================] - 15s 75ms/step - loss: 0.0187 - val_loss: 0.0202
Epoch 7/50
200/200 [==============================] - 16s 78ms/step - loss: 0.0181 - val_loss: 0.0194
Epoch 8/50
200/200 [==============================] - 15s 76ms/step - loss: 0.0178 - val_loss: 0.0192
Epoch 9/50
200/200 [==============================] - 15s 77ms/step - loss: 0.0175 - val_loss: 0.0183
Epoch 10/50
200/200 [==============================] - 16s 78ms/step - loss: 0.016

In [21]:
mse = model.evaluate(
    x = data_generator(
        1024,
        1000,
        1,
        mode = 'test'
    )
)

1000/1000 [==============================] - 275s 274ms/step - loss: 0.0155
